# MVVM + Hexagonal in SEVA — Part 2 (View ↔ ViewModel ↔ UseCase)

This part focuses on the **MVVM relationship** first, then briefly links back to adapters.

Goal:
1. Start from a **View-only** Tkinter example (no button business logic).
2. Add a **ViewModel** (UI state + command handling).
3. Add a **UseCase** (functional behavior, progress updates).
4. Show swapping **View** and **Adapter** without breaking VM logic.

In [ ]:
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

## 1) Step A — View only (no button logic)

The view builds widgets and renders state. The button is present, but no business logic is implemented yet.

In [ ]:
import tkinter as tk
from tkinter import ttk


class StartViewOnly:
    def __init__(self, root: tk.Tk):
        self.root = root
        self.root.title("MVVM Step A")

        self.status_var = tk.StringVar(value="Idle")
        self.progress_var = tk.DoubleVar(value=0.0)

        ttk.Label(root, text="Experiment status").pack(padx=8, pady=(8, 2), anchor="w")
        ttk.Label(root, textvariable=self.status_var).pack(padx=8, pady=2, anchor="w")
        ttk.Progressbar(root, maximum=100, variable=self.progress_var, length=260).pack(padx=8, pady=6)

        # Button exists, but no command logic yet.
        ttk.Button(root, text="Start").pack(padx=8, pady=(2, 8), anchor="w")

    def render(self, status_text: str, progress_pct: float) -> None:
        self.status_var.set(status_text)
        self.progress_var.set(progress_pct)

## 2) Step B — Add a ViewModel

Now the button click is forwarded to the ViewModel.
The ViewModel owns UI state and command handling, but still no external I/O.

In [ ]:
from dataclasses import dataclass


@dataclass
class StartVmState:
    status_text: str = "Idle"
    progress_pct: float = 0.0
    is_running: bool = False


class StartViewModelStepB:
    def __init__(self):
        self.state = StartVmState()

    def on_start_clicked(self) -> None:
        self.state.is_running = True
        self.state.status_text = "Started from ViewModel"
        self.state.progress_pct = 10.0

In [ ]:
class StartViewWithVm:
    def __init__(self, root: tk.Tk, vm: StartViewModelStepB):
        self.root = root
        self.vm = vm
        self.root.title("MVVM Step B")

        self.status_var = tk.StringVar(value="Idle")
        self.progress_var = tk.DoubleVar(value=0.0)

        ttk.Label(root, text="Experiment status").pack(padx=8, pady=(8, 2), anchor="w")
        ttk.Label(root, textvariable=self.status_var).pack(padx=8, pady=2, anchor="w")
        ttk.Progressbar(root, maximum=100, variable=self.progress_var, length=260).pack(padx=8, pady=6)
        ttk.Button(root, text="Start", command=self._on_start).pack(padx=8, pady=(2, 8), anchor="w")

        self.render()

    def _on_start(self):
        self.vm.on_start_clicked()
        self.render()

    def render(self):
        self.status_var.set(self.vm.state.status_text)
        self.progress_var.set(self.vm.state.progress_pct)

## 3) Step C — Add a UseCase (progress behavior)

The ViewModel now delegates behavior to a UseCase.
This keeps workflow logic out of the View and keeps ViewModel focused on state + commands.

In [ ]:
from seva.adapters.job_rest_mock import JobRestMock


class StartAndPollProgressUseCase:
    # Small teaching use case: start one mock plan and return progress snapshots.

    def __init__(self, job_port: JobRestMock):
        self.job_port = job_port
        self.active_group_id = None

    def start(self) -> str:
        plan = JobRestMock.example_plan()
        group_id, _runs = self.job_port.start_batch(plan)
        self.active_group_id = group_id
        return group_id

    def poll_progress_pct(self) -> float:
        if not self.active_group_id:
            return 0.0
        snapshot = self.job_port.poll_group(self.active_group_id)
        for box in snapshot.get("boxes", {}).values():
            runs = box.get("runs", [])
            if runs:
                return float(runs[0].get("progress_pct", 0.0))
        return 0.0

In [ ]:
class StartViewModel:
    def __init__(self, use_case: StartAndPollProgressUseCase):
        self.use_case = use_case
        self.state = StartVmState()

    def on_start_clicked(self) -> None:
        group_id = self.use_case.start()
        self.state.is_running = True
        self.state.status_text = f"Started group: {group_id}"

    def on_poll_clicked(self) -> None:
        progress = self.use_case.poll_progress_pct()
        self.state.progress_pct = progress
        self.state.status_text = f"Running... {progress:.1f}%"
        if progress >= 100.0:
            self.state.status_text = "Done"
            self.state.is_running = False

In [ ]:
class StartViewTk:
    # Simple Tkinter View: rendering + event wiring only.

    def __init__(self, root: tk.Tk, vm: StartViewModel):
        self.root = root
        self.vm = vm
        self.root.title("MVVM Step C")

        self.status_var = tk.StringVar(value="Idle")
        self.progress_var = tk.DoubleVar(value=0.0)

        ttk.Label(root, text="Status").pack(padx=8, pady=(8, 2), anchor="w")
        ttk.Label(root, textvariable=self.status_var).pack(padx=8, pady=2, anchor="w")
        ttk.Progressbar(root, maximum=100, variable=self.progress_var, length=260).pack(padx=8, pady=6)

        btn_row = ttk.Frame(root)
        btn_row.pack(padx=8, pady=(2, 8), anchor="w")
        ttk.Button(btn_row, text="Start", command=self._on_start).pack(side="left", padx=(0, 6))
        ttk.Button(btn_row, text="Poll", command=self._on_poll).pack(side="left")

        self.render()

    def _on_start(self):
        self.vm.on_start_clicked()
        self.render()

    def _on_poll(self):
        self.vm.on_poll_clicked()
        self.render()

    def render(self):
        self.status_var.set(self.vm.state.status_text)
        self.progress_var.set(self.vm.state.progress_pct)

## 4) Swap the View (same ViewModel)

To show modularity, here is a second view style with the same VM contract.

In [ ]:
class CompactStartViewTk:
    # Alternative UI skin using the same VM methods/state.

    def __init__(self, root: tk.Tk, vm: StartViewModel):
        self.root = root
        self.vm = vm
        self.root.title("Compact View")

        self.label = ttk.Label(root, text="Idle")
        self.label.pack(padx=8, pady=(8, 4))

        self.progress = ttk.Progressbar(root, maximum=100, length=220)
        self.progress.pack(padx=8, pady=4)

        ttk.Button(root, text="Start", command=self._start).pack(padx=8, pady=2)
        ttk.Button(root, text="Poll", command=self._poll).pack(padx=8, pady=(2, 8))
        self.render()

    def _start(self):
        self.vm.on_start_clicked()
        self.render()

    def _poll(self):
        self.vm.on_poll_clicked()
        self.render()

    def render(self):
        self.label.configure(text=self.vm.state.status_text)
        self.progress["value"] = self.vm.state.progress_pct

## 5) Cross-link to adapters (short)

The UI stays responsive and unchanged while adapters behind the use case are swapped.
- `JobRestMock()` for offline learning/testing.
- `JobRestAdapter(...)` for real API (e.g., Raspberry Pi-hosted service).
- `AmetekJobAdapter` for translating a different vendor API into the same `JobPort` method names.

The **JobPort method names** (`start_batch`, `poll_group`, ...) are the key contract.

In [ ]:
from seva.domain.ports import JobPort
from seva.domain.entities import ExperimentPlan


class AmetekJobAdapter(JobPort):
    def start_batch(self, plan: ExperimentPlan):
        print("AMETEK: translate experiment -> vendor payload")
        return ("ametek-demo-group", {})

    def cancel_run(self, box_id, run_id):
        print("AMETEK: cancel run", box_id, run_id)

    def cancel_runs(self, box_to_run_ids):
        print("AMETEK: cancel runs", box_to_run_ids)

    def cancel_group(self, run_group_id):
        print("AMETEK: cancel group", run_group_id)

    def poll_group(self, run_group_id):
        print("AMETEK: poll", run_group_id)
        return {"boxes": {}, "wells": []}

    def download_group_zip(self, run_group_id, target_dir):
        print("AMETEK: download", run_group_id, target_dir)
        return target_dir

## 6) Practical notebook flow

Try this in order:
1. Run Step A cells (view-only).
2. Run Step B cells (view + viewmodel).
3. Run Step C cells (view + vm + usecase) and click **Start**, then **Poll** multiple times.
4. Replace `StartViewTk` with `CompactStartViewTk` using the same VM.
5. Replace adapter inside the use case (`JobRestMock` vs another adapter) and observe that View/VM code does not need structural changes.